In [33]:
from IPython.core.display import display, HTML
display(HTML("<style>.container (width : 100% ! important: )</style>"))


import pandas as pd
import os
import re

In [34]:
# Labelling and Extract col
review_df = pd.read_csv("Booking.csv")
review_df["label"] = 1
review_df = review_df[["Positive", "label"]]
review_df.columns = ["Text", "Label"]
pos = review_df

review_df = pd.read_csv("Booking.csv")
review_df["label"] = 0
review_df = review_df[["Negative", "label"]]
review_df.columns = ["Text", "Label"]
neg = review_df

review_df = pd.concat([pos.dropna(), neg.dropna()])
review_df.tail()

,Text,Label
33242,도심과는 조금 멉니다,0
33243,약간 외곽에 있었고 가까이에 식당을 찾기가 어려웠어요,0
33244,방이 20m2이라고 했는데 실제론 절반크기(?) 정도로 많이 작았음...,0
33245,없음,0
33246,없습니다.,0


In [35]:
# cleansing with reg

del_filter = re.compile(r'[^ .,?!/@$%~％·∼()\x00-\x7F가-힣]+')

def filter_text(x):
    global del_filter
    return del_filter.findall(x)
review_df.Text.apply(filter_text)
review_df

,Text,Label
0,직원 분들이 정말로 친절하십니다. 외국인 친구를 데리고 갔는데 특히 아침 조식 때 ...,1
1,침대 매트리스는 좋았어요 아침식사도 좋구요,1
2,편의시설 가까움,1
3,위치가 좋다,1
4,화장실이 넓고 위치가 좋아요,1
...,...,...
33242,도심과는 조금 멉니다,0
33243,약간 외곽에 있었고 가까이에 식당을 찾기가 어려웠어요,0
33244,방이 20m2이라고 했는데 실제론 절반크기(?) 정도로 많이 작았음...,0
33245,없음,0


In [36]:
!pip install soynlp
!pip install tqdm

from tqdm.notebook import tqdm
from soynlp.normalizer import repeat_normalize

# 중복되는 글자 제거

review_df.index = [i for i in range(review_df["Text"].size)]

for idx in tqdm(review_df.index):
    review_df["Text"][idx] = repeat_normalize(review_df["Text"][idx], num_repeats=2)

review_df

<ipython-input-36-11d5dce1a42a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df["Text"][idx] = repeat_normalize(review_df["Text"][idx], num_repeats=2)


,Text,Label
0,직원 분들이 정말로 친절하십니다. 외국인 친구를 데리고 갔는데 특히 아침 조식 때 ...,1
1,침대 매트리스는 좋았어요 아침식사도 좋구요,1
2,편의시설 가까움,1
3,위치가 좋다,1
4,화장실이 넓고 위치가 좋아요,1
...,...,...
66305,도심과는 조금 멉니다,0
66306,약간 외곽에 있었고 가까이에 식당을 찾기가 어려웠어요,0
66307,방이 20m2이라고 했는데 실제론 절반크기(?) 정도로 많이 작았음...,0
66308,없음,0


In [37]:
# neg, pos division
neg = review_df[review_df["Label"] == 0]["Text"]
pos = review_df[review_df["Label"] == 1]["Text"]

In [38]:
# kss divide with sentences
import kss

def split_sentence(texts_df):
    arr = []
    for sentence in tqdm(texts_df):
        for s in kss.split_sentences(sentence):
            arr.append(s)
    return arr


neg = pd.DataFrame({"Text": split_sentence(neg)})
pos = pd.DataFrame({"Text": split_sentence(pos)})

neg["Label"] = 0
pos["Label"] = 1

# del record that have only 1 word

review_splited = pd.concat([pos, neg])
review_splited["len_text"] = [len(t.split()) for t in review_splited["Text"]]
review_splited_del_meaningless = review_splited[review_splited["len_text"] > 1]

# flushing idx

review_splited_del_meaningless.index = [i for i in range(review_splited_del_meaningless["Text"].size)]

In [39]:
review_splited_del_meaningless

,Text,Label,len_text
0,직원 분들이 정말로 친절하십니다.,1,4
1,외국인 친구를 데리고 갔는데 특히 아침 조식 때 직원분이 많이 도와주셔서 정말로 간...,1,13
2,그리고 데스크에 계신 직원분들도 체크 아웃하고 짐도 맡겨주시고 정말로 친절하셨습니다.,1,10
3,방은 깨끗하고 특히 화장실이 청결했어요.,1,5
4,약간 작은 방이긴 하지만 위치도 명동역에서 2분 거리라 좋았습니다.,1,9
...,...,...,...
105014,호텔에 대해선 아무 불만도 없습니다.,0,5
105015,다만 숙소 근처에 맛집이 없었던게 아쉬웠습니다. 가장 가까운 지하철역까지도 약간 거...,0,17
105016,도심과는 조금 멉니다,0,3
105017,약간 외곽에 있었고 가까이에 식당을 찾기가 어려웠어요,0,7


In [42]:
review_splited_del_meaningless.to_csv("Preprocessing_review_pos_neg.csv",
                                     index = False)

In [43]:
pd.read_csv("Preprocessing_review_pos_neg.csv")

,Text,Label,len_text
0,직원 분들이 정말로 친절하십니다.,1,4
1,외국인 친구를 데리고 갔는데 특히 아침 조식 때 직원분이 많이 도와주셔서 정말로 간...,1,13
2,그리고 데스크에 계신 직원분들도 체크 아웃하고 짐도 맡겨주시고 정말로 친절하셨습니다.,1,10
3,방은 깨끗하고 특히 화장실이 청결했어요.,1,5
4,약간 작은 방이긴 하지만 위치도 명동역에서 2분 거리라 좋았습니다.,1,9
...,...,...,...
105014,호텔에 대해선 아무 불만도 없습니다.,0,5
105015,다만 숙소 근처에 맛집이 없었던게 아쉬웠습니다. 가장 가까운 지하철역까지도 약간 거...,0,17
105016,도심과는 조금 멉니다,0,3
105017,약간 외곽에 있었고 가까이에 식당을 찾기가 어려웠어요,0,7
